<a href="https://colab.research.google.com/github/JCaballerot/Recommender_Systems/blob/main/Pearson_Similarity_Recommender/MovieLens_PS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


<h1 align=center><font size = 5> Pearson similarity Recommender</font></h1>

---

<center>
  <img src="https://bobliu.io/assets/img/cards.509a5045.jpg" width="800" height="300">
</center>


## Objetivo de este Notebook

1. Cargar y preprocesar un Dataset.
2. Realizar un sistema de recomendación basado en Pearson Similarity.
3. Comprobar el performance del sistema.

## Tabla de Contenidos

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>
    
1. <a href="#item31">Contexto</a>  
2. <a href="#item32">Descargar y preparar el Dataset</a>  
6. <a href="#item34">Entrenamiento del modelo</a>  
6. <a href="#item34">Validación del modelo</a>  

</font>
</div>

## 1. Contexto


El conjunto de datos MovieLens es uno de los conjuntos de datos de recomendación más populares y ampliamente utilizados en la investigación de sistemas de recomendación. Fue creado por el GroupLens Research Project en la Universidad de Minnesota para impulsar la investigación en sistemas de recomendación, proporcionando un recurso valioso para la comunidad académica y promoviendo el desarrollo y la comprensión de tecnologías de recomendación personalizada.


<b>Descripción de datos</b>

El conjunto de datos MovieLens contiene información sobre:

<b>Películas:</b> Detalles sobre las películas, incluyendo su título, género y año de lanzamiento.

<b>Usuarios:</b> Perfiles de los usuarios que han calificado y/o etiquetado las películas, incluyendo su ID y otros detalles demográficos opcionales.

<b>Calificaciones:</b> Calificaciones numéricas que los usuarios asignan a las películas en una escala de 1 a 5.

<b>Etiquetas:</b> Palabras clave o tags proporcionados por los usuarios para describir el contenido o la esencia de las películas.

El conjunto de datos es ampliamente utilizado con fines académicos y de investigación, siendo una referencia en el diseño y evaluación de sistemas de recomendación de películas. También es útil para el análisis de tendencias y comportamientos en la visualización de películas y la interacción del usuario con el contenido.

<strong>Puede consultar este [link](https://grouplens.org/datasets/movielens/) para leer más sobre la fuente de datos MovieLens proporcionada por GroupLens Research en la Universidad de Minnesota.</strong>

## 2. Descargar y preparar Dataset

In [ ]:
# Descargar el dataset Movielens
!curl -o dataset.zip "https://files.grouplens.org/datasets/movielens/ml-latest-small.zip"
!unzip dataset.zip
!ls -la

In [8]:
# Principales librerías
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore") # Turn off warnings


In [9]:
links   = pd.read_csv("ml-latest-small/links.csv")
movies  = pd.read_csv("ml-latest-small/movies.csv")
ratings = pd.read_csv("ml-latest-small/ratings.csv")
tags    = pd.read_csv("ml-latest-small/tags.csv")


In [ ]:
links.head()

In [ ]:
movies.head()

In [ ]:
ratings.head()

In [ ]:
tags.head()

In [14]:
print("  Movies: {} \n  Ratings: {}".format(len(movies), len(ratings)))


  Movies: 9742 
  Ratings: 100836


In [15]:
# Fusiona ambos datasets basados en la columna 'movieId'
data = pd.merge(ratings, movies, on='movieId')

In [16]:
movie_titles = data['title'].unique().tolist()
movie_ids = data['movieId'].unique().tolist()


In [70]:
# Crear matriz pivotada de usuarios y películas
user_movie_rating = data.pivot_table(index='userId', columns='title', values='rating')


In [73]:
#500 películas más vistas
movies_pop = user_movie_rating.isnull().sum().sort_values()[:500]


In [121]:
user_movie_rating = user_movie_rating[movies_pop.index.tolist()]

In [122]:
user_movie_rating = user_movie_rating.reset_index()

Muestreo

In [ ]:
user_movie_rating

In [124]:
from sklearn.model_selection import train_test_split

# Convertir la matriz pivotada en un DataFrame y dividir en train y test
train_data, test_data = train_test_split(user_movie_rating, test_size=0.2, random_state=42)


In [125]:
# Volver a crear matrices pivotadas para entrenamiento y prueba
train_data_matrix = train_data.fillna(0)
test_data_matrix = test_data.fillna(0)

In [ ]:
test_data_matrix.head()

## 3. Pearson Similarity

Aplicaremos el enfoque de similaridad de pearson con enfoque ítem-ítem

In [197]:
# Calcular la matriz de correlación de Pearson entre las películas en el conjunto de entrenamiento
movie_correlation = train_data_matrix.corr()
movie_correlation = movie_correlation.reset_index()

In [ ]:
movie_correlation.shape

In [ ]:
# Seleccionar un usuario (por ejemplo, el usuario con ID 5)
user_id = 82

# Películas que el usuario ha visto
user_movies = data[data['userId'] == user_id]
watched_movies = user_movies[['title', 'genres', 'rating']]
watched_movies.sort_values(by = 'rating', ascending = False, inplace = True)

print(f"Películas vistas por el usuario {user_id}:")
watched_movies.head(5)

In [ ]:
movie_correlation

In [ ]:
# Porque te gustó "Lethal Weapon 2" ....
movie_correlation[['title', 'Lethal Weapon 2 (1989)']].sort_values(by = 'Lethal Weapon 2 (1989)', ascending = False)[1:].head(3)

In [ ]:
# Porque te gustó "Terminator 2" ....
movie_correlation[['title', 'Terminator 2: Judgment Day (1991)']].sort_values(by = 'Terminator 2: Judgment Day (1991)', ascending = False)[1:].head(3)

In [ ]:
# Porque te gustó "Indiana Jones and the Temple of Doom" ....
movie_correlation[['title', 'Indiana Jones and the Temple of Doom (1984)']].sort_values(by = 'Indiana Jones and the Temple of Doom (1984)', ascending = False)[1:].head(3)

In [164]:
%%capture
!pip install adjustText

PCA para entender la similaridad

In [195]:
# Crear matriz pivotada de usuarios y películas
user_movie_rating = data.pivot_table(index='userId', columns='title', values='rating')

#30 películas más vistas
movies_pop = user_movie_rating.isnull().sum().sort_values()[:50]

user_movie_rating = user_movie_rating[movies_pop.index.tolist()]
user_movie_rating = user_movie_rating.reset_index()

movie_correlation = user_movie_rating.fillna(0).corr()
movie_correlation = movie_correlation.reset_index()


In [ ]:
from sklearn.decomposition import PCA
from adjustText import adjust_text

# Aplicar PCA
pca = PCA(n_components=2)
movie_pca = pca.fit_transform(movie_correlation.drop(columns=['title', 'userId']))

# Obtener las n películas más vistas
top_30_movies = user_movie_rating.count().sort_values(ascending=False)[:50].index.tolist()
top_30_movies = list(set(top_30_movies)-set(['title', 'userId']))

# Graficar las n películas más vistas usando PCA
plt.figure(figsize=(20, 10))
for title in top_30_movies:
    idx = movie_correlation[movie_correlation['title'] == title].index[0]
    plt.scatter(movie_pca[idx, 0], movie_pca[idx, 1], s=50)
    plt.text(movie_pca[idx, 0], movie_pca[idx, 1], title, fontsize=7)

# Evitar superposición de etiquetas (si tienes adjustText instalado)
#adjust_text(texts, arrowprops=dict(arrowstyle='->', color='red'))

plt.xlabel('PCA 1')
plt.ylabel('PCA 2')
plt.title('Visualización de la Similaridad de las 30 Películas Más Vistas usando PCA')
plt.show()

## 4. Evaluación del modelo

In [215]:

def recommend_movies(data, user_id, movie_correlation, n_movies=3):
    # 1. Obtener las películas vistas por el usuario y sus calificaciones
    user_ratings = data[data['userId'] == user_id]

    # 2. Ordenar por calificación y tomar las n_movies películas mejor calificadas
    top_user_movies = user_ratings.sort_values(by='rating', ascending=False)['title'].head(n_movies)

    recommended = []
    for movie in top_user_movies:
        # 3. Obtener películas similares basadas en la matriz de correlación
        # Nos aseguramos de que el título de la película exista en la matriz de correlación antes de obtener recomendaciones.
        if movie in movie_correlation.columns:
            similar_movies = movie_correlation[movie].sort_values(ascending=False).index[1:n_movies+1].tolist()
            recommended.extend(similar_movies)

    # 4. Remover duplicados y retornar recomendaciones
    return list(set(recommended))

In [ ]:
user_id_sample = 83  # ID de usuario
recommendations = recommend_movies(data, user_id_sample, movie_correlation)
print(recommendations)


In [ ]:
movies[movies.movieId.isin(recommendations)]

Evaluación del hit Rate

In [231]:
def recommend_most_correlated_for_user(user_ratings, movie_correlation):
    recommended_movies = []
    for movie, rating in user_ratings.iteritems():
        if not np.isnan(rating) and movie in movie_correlation.columns:
            most_correlated = movie_correlation[movie].sort_values(ascending=False).index[1]
            recommended_movies.append(most_correlated)
    return recommended_movies

# Aplicar la función a cada fila (usuario) en test_data
test_data['recommended_movies'] = test_data.apply(lambda row: recommend_most_correlated_for_user(row, movie_correlation), axis=1)

In [ ]:
def compute_hit_rate(test_data):
    hits = 0
    total_recommendations = 0
    for index, row in test_data.iterrows():
        for recommended_movie in row['recommended_movies']:
            if not np.isnan(row[recommended_movie]):
                hits += 1
            total_recommendations += 1
    hit_rate = hits / total_recommendations if total_recommendations != 0 else 0
    return hit_rate

hit_rate = compute_hit_rate(test_data)
print(f"Hit Rate: {hit_rate:.4f}")

In [ ]:
# Calculando hit rate para test
test_hit_rate = hit_rate(train_data_matrix.values, movie_similarity, test_data_matrix.values)
print(f"Hit Rate (Test): {test_hit_rate:.4f}")

MAE, RMSE

In [ ]:
# Predicciones para cada película que el usuario ha calificado
def compute_predictions(row, movie_correlation):
    predictions = {}
    for movie in row.dropna().index:
        if movie != 'recommended_movies':
            predicted_rating = predict_rating(row, movie, movie_correlation)
            if not np.isnan(predicted_rating):
                predictions[movie] = predicted_rating
    return predictions

test_data['predictions'] = test_data.apply(lambda row: compute_predictions(row, movie_correlation), axis=1)

# Calcular los errores para las predicciones
def compute_errors(row):
    errors = {}
    for movie, predicted_rating in row['predictions'].items():
        real_rating = row[movie]
        if not np.isnan(real_rating):
            errors[movie] = real_rating - predicted_rating
    return errors



In [ ]:
test_data['errors'] = test_data.apply(compute_errors, axis=1)

# Calcular MAE y RMSE
all_errors = [error for movie_errors in test_data['errors'] for error in movie_errors.values()]

mae = np.mean(np.abs(all_errors))
rmse = np.sqrt(np.mean(np.square(all_errors)))

print(f"MAE: {mae:.4f}")
print(f"RMSE: {rmse:.4f}")

---
## Gracias por completar este laboratorio!